In [2]:
import dao.MongoTagUser as mtu
import panda.TagUser as tupd

mongotu = mtu.TagofSongUserRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# list = mongotu.makeRecomUsersSet()
# print(list)

In [3]:
doc = mongotu.rdb.findDocument(
            collection_name="like", query=mongotu.query, projection=mongotu.projections)
# 储存请求用户的名字
mongotu.user_name = doc[0]['name']
# 随机获取一定数量的用户来计算tag 相似度
# 统一获取其他用户的信息
querys = [{'$sample': {"size": 50-1}}]
project = {'$project': mongotu.a_projections}
querys.append(project)
docs = mongotu.rdb.aggregateDocument(
     collection_name="like", querys=querys)
print(doc)

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'tags': ['爵士', '驾车', '古典', '90后', '民族', '伤感', '安静', '钢琴', '雷鬼', '学习', '怀旧', '夜晚', '拉丁', '轻音乐', '英伦', '治愈', 'R&B/Soul', '快乐', '孤独', '放松', 'Bossa Nova', '古风', '说唱', '游戏', '金属', '小语种', '吉他', '粤语', '兴奋', '舞曲', '思念', '影视原声', '朋克', '清新', '经典', '后摇', 'New Age', '日语', '流行', '乡村', '性感', '酒吧', '蓝调', '世界音乐', '民谣', '韩语', '欧美', '摇滚', '另类/独立', '浪漫', '感动', '电子', 'ACG', '旅行', '下午茶', '华语', '器乐', '运动']}]


In [4]:
# 判断 docs中是否存在 doc 并合并
docs = mongotu.isDocinDocs(doc=doc, docs=docs)
# print(docs)     

In [5]:
# 查找数据songs 便于 统计tags 出现的次数
# 提取docs 中用户的id，之后用
list_name = mongotu.getItemFromDocs(docs=docs, key='name')
list_id = mongotu.getItemFromDocs(docs=docs, key='id')
# 通过这个方法获取songs，来计算tag出现次数
docs_song = mongotu.getSongsforTagfromMongo(list_id)
# count_tag 统计
dict_tags = mongotu.makeMatricTagCountData(docs=docs_song)
# print(dict_tags)
        

In [6]:
# 根据以上docs 来生成 xy轴，矩阵
# 内容
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongotu.makeXYMatric(docs=docs)
# print(data)
        

In [7]:
# 生成pandas 表格
tudf = tupd.TUPandas(datas=list(
map(list, data.toarray())), items=mongotu.y, users=mongotu.x)



In [8]:
tudf.df 

,RuRuUIH,BJ1HR0,j,h,s7UIjlHxS,m3NOW7Pp,EQAFph8,zhZd1J1,S0JlP3z,6p11UivDhF,...,i7qZ,ud3K,CO0y,eVbz,PdHcku8A,ekkYWd,b,BrQ,ere,xtbElzcA
性感,1,1,1,0,1,1,1,1,1,1,...,1,0,1,1,1,0,0,0,1,0
民谣,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,0
钢琴,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
思念,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
放松,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
学习,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,0
摇滚,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
酒吧,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,0
古典,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,0


In [9]:
# 开始将tudf 中的数据根据上面的dict_tags中的数据进行替换
tudf = mongotu.changeDataCountTagTUDF(list_name, dict_tags, tudf)
   

In [10]:
tudf.df     

,RuRuUIH,BJ1HR0,j,h,s7UIjlHxS,m3NOW7Pp,EQAFph8,zhZd1J1,S0JlP3z,6p11UivDhF,...,i7qZ,ud3K,CO0y,eVbz,PdHcku8A,ekkYWd,b,BrQ,ere,xtbElzcA
性感,1,3,1,0,3,5,3,2,3,2,...,1,0,1,1,3,0,0,0,2,0
民谣,26,17,27,4,30,28,15,28,40,32,...,19,7,13,10,34,0,21,10,19,0
钢琴,29,32,31,5,35,58,28,44,40,57,...,30,10,18,23,24,0,23,11,23,2
思念,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
放松,10,9,7,1,13,16,14,24,20,20,...,9,7,8,12,6,0,11,3,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
学习,20,17,15,6,23,20,15,28,19,20,...,12,11,14,6,18,0,18,7,10,0
摇滚,79,65,71,17,86,114,77,117,121,112,...,72,48,50,52,58,1,58,35,68,9
酒吧,3,3,4,0,3,3,4,4,7,4,...,2,3,1,3,2,0,3,0,1,0
古典,36,21,31,7,32,39,27,42,44,40,...,21,15,18,15,32,0,21,7,23,0


In [11]:
# 推荐过程
similar_user = tudf.makeSimilarityWithPearson()

In [12]:
similar_user

,RuRuUIH,BJ1HR0,j,h,s7UIjlHxS,m3NOW7Pp,EQAFph8,zhZd1J1,S0JlP3z,6p11UivDhF,...,i7qZ,ud3K,CO0y,eVbz,PdHcku8A,ekkYWd,b,BrQ,ere,xtbElzcA
RuRuUIH,1.00,0.97,0.96,0.94,0.97,0.98,0.97,0.98,0.97,0.98,...,0.96,0.95,0.97,0.96,0.96,0.38,0.97,0.96,0.96,0.92
BJ1HR0,0.97,1.00,0.95,0.93,0.98,0.98,0.97,0.98,0.98,0.98,...,0.96,0.96,0.96,0.95,0.97,0.40,0.97,0.96,0.97,0.91
j,0.96,0.95,1.00,0.93,0.97,0.97,0.97,0.98,0.98,0.97,...,0.98,0.94,0.97,0.95,0.97,0.52,0.96,0.97,0.97,0.90
h,0.94,0.93,0.93,1.00,0.93,0.94,0.95,0.94,0.94,0.93,...,0.94,0.93,0.95,0.91,0.93,0.39,0.93,0.93,0.96,0.89
s7UIjlHxS,0.97,0.98,0.97,0.93,1.00,0.98,0.98,0.98,0.98,0.99,...,0.98,0.95,0.96,0.96,0.97,0.44,0.97,0.97,0.97,0.92
m3NOW7Pp,0.98,0.98,0.97,0.94,0.98,1.00,0.98,0.99,0.99,0.99,...,0.98,0.95,0.97,0.96,0.97,0.41,0.97,0.97,0.97,0.92
EQAFph8,0.97,0.97,0.97,0.95,0.98,0.98,1.00,0.98,0.98,0.98,...,0.97,0.96,0.97,0.96,0.97,0.45,0.97,0.98,0.98,0.93
zhZd1J1,0.98,0.98,0.98,0.94,0.98,0.99,0.98,1.00,0.99,0.99,...,0.98,0.97,0.97,0.97,0.97,0.44,0.98,0.98,0.97,0.94
S0JlP3z,0.97,0.98,0.98,0.94,0.98,0.99,0.98,0.99,1.00,0.98,...,0.97,0.95,0.97,0.95,0.98,0.43,0.97,0.98,0.98,0.92
6p11UivDhF,0.98,0.98,0.97,0.93,0.99,0.99,0.98,0.99,0.98,1.00,...,0.98,0.96,0.97,0.96,0.97,0.44,0.97,0.97,0.97,0.92


In [13]:
# 获取相似度最高几位用户sign 设置为 1 表示 使用Pearson相关系数获取相关性最高的几个用户
diction = tudf.makeTopNUsers(similar=similar_user,sign=1)
print(diction)
print(len(diction['RuRuUIH']))

{'RuRuUIH': ['m3NOW7Pp', 'zhZd1J1', '6p11UivDhF', '1PufH', 'ReOewQ', 'OejC0', 'X2Bwh38', 'BJ1HR0', 'dj775w', 'zFpXHZ0n', 'HCV4F', 'J94COPc', '2ILOvc', 'IDDRfNJpw', 'CO0y', 'H298YN', 'FnJi', 'b', 'zVZzhaw', 'S0JlP3z', 'EQAFph8', 's7UIjlHxS', 'eVbz', 'PdHcku8A', 'i7qZ', 'BrQ', 'qLy5bnF', 'ere', '79vexmSD', 'c4D', 'afqmIx79w', 'j', 'dyrvSYZxx', '1C3ijya', 'MGrVl', 'G']}
36


In [14]:
# 利用当前用户的信息确定他们的id值形成对应的list_id
diction[mongotu.user_name] = mongotu.makeListIdWithUserName(diction[mongotu.user_name])
print(diction)
print(len(diction[mongotu.user_name]))

{'RuRuUIH': ['1a78ba0cb67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78d9f6b67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a78dab4b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a790d2cb67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78c74ab67911ed95dd00155dadb10b', '1a78e130b67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a790b06b67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78af76b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a79073cb67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78bfcab67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a78dd66b67911ed95dd00155dadb10b', '1a78ca38b67911ed95dd00155dadb10b', '1a78cd12b67911ed95dd00155dadb10b', '1a78f3f0b67911ed95dd00155dadb10b', '1a78f274b67911ed95dd00155dadb10b', '1a78d0fab67911

In [15]:
# 保存过程
# mongotu.saveDataSetRecomAnswer(diction)
# 改变形式，方便存储
docs = mongotu.changeDataFormat(diction=diction, list_keys=list(diction))
print(docs)
# 保存
mongotu.wdb.writeDocument(docs=docs, collection_name="recom")

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'recomUsers': ['1a78ba0cb67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78d9f6b67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a78dab4b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a790d2cb67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78c74ab67911ed95dd00155dadb10b', '1a78e130b67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a790b06b67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78af76b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a79073cb67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78bfcab67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a78dd66b67911ed95dd00155dadb10b', '1a78ca38b67911ed95dd00155dadb10b', '1a78cd12b67911ed95dd00155dadb10b', '1a78f3f0b67911ed95dd0